# Project Report: bci4als
#### Authors: Evyatar Luvaton, Noam Siegel

This software was developed for the course Brain-Computer-Interface for ALS Patients, December 2020.

Over the mid-semester project we have integrated the different parts of BCI which been discussed in the course.
We introduce bci4als, a complete pipeline for EEG motor imagery data recording and classification.

The pipeline is general, so it can be extended to other BCI settings like P300, SSVEP.

The first part of this report presents the python package bci4als.


## Preliminaries

First, go ahead and install bci4als (if you haven't already done so).

In [1]:
# !pip install -i https://test.pypi.org/simple/ bci4als;

Looking in indexes: https://test.pypi.org/simple/


The main codebase lies in the `bci4als.MI` module:

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import mne
import pickle
import bci4als.mi as mi


C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


ModuleNotFoundError: No module named 'bci4als.mi.test_model'

## Part 1 - End-to-End Walkthrough

We will now guide you how to use the `bci4als` pipeline from end to end.

<B>Note:</B>
All the scripts share common configuration parameters, which are stored in a multi-level `dict` object called `mi.params`. The first-layer keys indicate the category of parameters:

In [ ]:
params = mi.params
params.keys()

For example, to see the parameters related to preprocessing:

In [ ]:
preprocess_params = params['preprocess']
print('\n'.join('{}: {}'.format(k, v) for k, v in preprocess_params.items()))

### Step 1 - Record Experiment
The `bci4als.mi.record` module is responsible for executing a Motor Imagery experiment and recording the stimulus.

The `bci4als.mi.record.start()` function starts an interactive GUI which will guide you through executing a MI experiment. It assumes you:

1) have an external application streaming EEG data (via LSL) to Lab Recorder.

2) have a `CurrentStudy` directory into which the collected data will be saved.

3) opened `LabRecorder`.

Once you are ready, begin recording:|

In [ ]:
# call this method to start a recording:
# mi.record.start()

Now follow the following steps:

1) You should see a "Welcome" messagebox. Press "OK" to continue.

2) Select the CurrentStudy folder.

3) Type a new Session ID (this will be the name of the folder). Press "OK" to continue.

4) You should a confirmation messabox. Press "OK" to continue.

5) Point the LabRecorder to the folder created at step 3. Set the name of the file EEG.xdf

6) <B> Make sure you are prepared to begin the motor imagery experiment. </B>Press "OK" to begin the trials.

7) After the experiment, press "OK" to close the program.


Lab Recorder will generate the `EEG.xdf` file which contains the EEG and markers recording. It can be read with pyxdf.

Python will generate `stimulus_vector.csv`, which contains the labels for each trial.


### Step 2 - Data Preprocessing

The `preprocess.py` script is responsible for cleaning the data.

Currently, the pre-processing script uses low-pass, high-pass and notch filters to clean the data.

The parameters for the filters are also part of the `params` object:

In [ ]:
params['preprocess']['filter']

The script will output an `EEG_clean.csv` file, which contains the EEG data
after the pre-processing. Additionally, the script exports a `.info` json file. The `.info` file
contains all the info about the EEG stream. `extract_features.py` uses the info of the streaming.
The output is automatically saved in the data directory.

Let's plot the clean EEG data:

In [ ]:
#------- todo: find out if this is a normal psd plot ------

# Params
eeg_clean_path = '../data/noam/2/EEG_clean.csv'
ch_names = ['C03', 'C04', 'P07', 'P089', 'O01', 'O02', 'F07', 'F08', 'F03', 'F04', 'T07', 'T08', 'P03']
s_rate = 125

# Create mne info & raw
eeg_clean = np.genfromtxt(eeg_clean_path, delimiter=',', skip_header=1)[:, 1:]
info = mne.create_info(ch_names, s_rate, verbose=False)
raw_eeg_clean = mne.io.RawArray(eeg_clean.T, info, verbose=False)

# Plot
fig = raw_eeg_clean.plot_psd(picks=ch_names, show=False)

### Step 3 - Data Segmentation

In the `segment_data.py` script we split the data for each trial.
The start and end of each trial is according to the markers streaming we created
while recording the EEG. The script gets the subject folder from the config file
(under `config['data']['subject_folder']`) and splits for trial each
EEG record in each day.

The output of MI3 script is a pickle file named `EEG_trials.pickle` which
located in the corresponded day directory. The file is a list with ndarray
where each ndarray is the corresponded EEG data of the trial.

In [ ]:
eeg_trials_path = '../data/noam/3/EEG_trials.pickle'
eeg_trials = pickle.load(open(eeg_trials_path, 'rb'))

print('Number of trials: {}\nTrials dimensions: {}'.format(len(eeg_trials), eeg_trials[0].shape))

And a PSD plot of a specific trial look like:


In [ ]:
eeg_trial = mne.io.RawArray(eeg_trials[103].T, info, verbose=False)
fig = eeg_trial.plot_psd(picks=ch_names)

### Step 4 - Feature Extraction


### Step 5 - Train and Test a Model
mi.ass

## Part 2: Experiment with ResNET Features
In this part we use the RESNet Feature Extraction.